In [1]:
#!/usr/bin/env python
# coding: utf-8
# 필요 라이브러리 가져오기
import cv2
import numpy as np

import os
import glob

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, Conv2D, Activation, MaxPool2D, Flatten, Dense, Add
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# 모델에 사용되는 이미지 크기 설정 및 데이터 분류 개수
input_height = 48
input_width = 48
input_channel = 3

input_shape = (input_height, input_width, input_channel)
n_classes = 5  # left, stop, forward, school, noSign 

# 인공지능 학습에 사용되는 이미지 데이터 위치 지정
data_dir = './data'
data1_path = os.path.join(data_dir, 'Left', '*.jpg')
data1_files = glob.glob(data1_path)
data2_path = os.path.join(data_dir, 'Forward', '*.jpg')
data2_files = glob.glob(data2_path)
data3_path = os.path.join(data_dir, 'Stop', '*.jpg')
data3_files = glob.glob(data3_path)
data4_path = os.path.join(data_dir, 'schoolZone', '*.jpg')
data4_files = glob.glob(data4_path)
data5_path = os.path.join(data_dir, 'noSign', '*.jpg')
data5_files = glob.glob(data5_path)

test_path = os.path.join(data_dir, 'Test', '*.jpg')
test_files = glob.glob(test_path)

In [4]:
# 인공지능 학습에 사용되는 이미지의 총 개수 계산
n_train = len(data1_files) + len(data2_files) + len(data3_files) + len(data4_files) + len(data5_files)
n_test = len(test_files)
print(n_train)
print(n_test)

0
0


In [5]:
# 인공지능 학습 데이터 및 레이블이 들어갈 변수 초기화
trainset = np.zeros(
    shape=(n_train, input_height, input_width, 
           input_channel), dtype='float32')

label = np.zeros(
    shape=(n_train, n_classes), dtype='float32')

testset = np.zeros(
    shape=(n_test, input_height, input_width, 
           input_channel), dtype='float32')

In [6]:
# 인공지능 학습에 사용될 이미지 데이터 파일 전체
train_files = data1_files + data2_files + data3_files + data4_files + data5_files
# 인공지능 학습에 사용될 이미지 데이터 크기 조정
# 훈련데이터 셋에 저장
for ind, file in enumerate(train_files):
    try:
        image = cv2.imread(file)
        resized_image = cv2.resize(image,
                        (input_width, input_height))
        trainset[ind] = resized_image
    except Exception as e:
        print(file)

# 인공지능 학습 후 평가를 위해 사용될 이미지 데이터 크기 조정
# 평가데이터 셋에 저장
for ind, file in enumerate(test_files):
    try:
        image = cv2.imread(file)
        resized_image = cv2.resize(image,
                            (input_width, input_height))
        testset[ind] = resized_image
    except Exception as e:
        print(file)

# 훈련데이터 및 평가데이터를 0과 1 수로 변환
trainset = trainset / 255.0
testset = testset / 255.0

In [7]:
# 이미지 신호별 데이터 파일 전체 개수 저장
n_data1 = len(data1_files)
n_data2 = len(data2_files)
n_data3 = len(data3_files)
n_data4 = len(data4_files)
n_data5 = len(data5_files)


# 이미지 신호별 레이블 값 정의
begin_ind = 0
end_ind = n_data1
label[begin_ind:end_ind, 0] = 1.0

begin_ind = n_data1
end_ind = n_data1 + n_data2
label[begin_ind:end_ind, 1] = 1.0

begin_ind = n_data1 + n_data2
end_ind = n_data1 + n_data2 + n_data3
label[begin_ind:end_ind, 2] = 1.0

begin_ind = n_data1 + n_data2 + n_data3
end_ind = n_data1 + n_data2 + n_data3 + n_data4
label[begin_ind:end_ind, 3] = 1.0

begin_ind = n_data1 + n_data2 + n_data3 + n_data4
end_ind = n_data1 + n_data2 + n_data3 + n_data4 + n_data5
label[begin_ind:end_ind, 4] = 1.0

In [8]:
# CNN 모델을 만드는 함수
def custom_model(input_shape, n_classes):

    input_tensor = Input(shape=input_shape)

    x = conv_block(input_tensor, 32)
    x = conv_block(x, 64)
    x = conv_block(x, 128)
    x = conv_block(x, 256)
    x = conv_block(x, 512)

    x = Flatten() (x)
    x = BatchNormalization() (x)
    x = Dense(512, activation='relu') (x)
    x = Dense(512, activation='relu') (x)

    output_layer = Dense(n_classes, activation='softmax') (x)

    inputs = [input_tensor]
    model = Model(inputs, output_layer)

    return model

In [9]:
# CNN 모델을 만드는 함수
def conv_block(x, filters):
    x = BatchNormalization() (x)
    x = Conv2D(filters, (3, 3), activation='relu', padding='same') (x)

    x = BatchNormalization() (x)
    shortcut = x
    x = Conv2D(filters, (3, 3), activation='relu', padding='same') (x)
    x = Add() ([x, shortcut])
    x = MaxPool2D((2, 2), strides=(2, 2)) (x)

    return x

In [10]:
# 메인함수
def main():
    
    # 모델 함수 호출
    model = custom_model(input_shape, n_classes)
    
    # 인공지능 모델을 이용한 최적 데이터 학습 방법 설계
    adam=Adam()
    model.compile(
        optimizer=adam,
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    # 인공지능 모델을 이용한 데이터 학습 실행
    history = model.fit(
        trainset, label, 
        batch_size=20, 
        epochs=20,  
        validation_split=0.005)
            
    # 인공지능 학습 후 모델 파일(model.json) 저장
    # 가중치 값 파일(weights.h5) 저장
    model_desc = model.to_json()
    with open('./model/model.json', 'w') as file_model:
        file_model.write(model_desc)
    model.save_weights('./model/weights.h5')

    # 평가데이터를 이용한 예측 정확도 확인
    if testset.shape[0] != 0:
        result_onehot = model.predict(testset)
        result_predict = np.argmax(result_onehot, axis=1)
    else:
        result_sparse = list()

    print('File name\t forecast category')

    # 평가 데이터 파일명에 따른 예측 결과 값 출력
    for file, label_id in zip(test_files, result_predict):
        
        filename = os.path.basename(file)
        
        if label_id == 0:
            label_name = 'Left'
        elif label_id == 1:
            label_name = 'Forward'
        elif label_id == 2:
            label_name = 'Stop'
        elif label_id == 3:
            label_name = 'School Zone'
        elif label_id == 4:
            label_name = 'noSign'

        print('%s\t%s' % (filename, label_name))
        
    # 모델 요약 및 학습 진행 결과 출력
    model.summary()
    
if __name__ == '__main__':
    main()

NameError: name 'Input' is not defined